In [2]:
import numpy
import xmltodict
from pythreejs import *
import pythreejs.enums

In [3]:
with open("Opalaye.jml") as fd:
    paths = xmltodict.parse(fd.read())

events = sorted(paths['jml']['pattern']['event'], key=lambda event: event['@t'])

trajectories = {'1':[], '2':[], '3':[]}

for event in events:
    position = numpy.array([event['@t'], event['@x'], event['@z'], event['@y']])
    for action_name in ['throw', 'holding','catch']:
        if action_name in event:
            action = event[action_name]
            if isinstance(action, list):
                paths = [a['@path'] for a in action]
            else:
                paths = action['@path']
            for path in paths:
                if action=='catch' and trajectories[path]:
                    previous_position = trajectories[path][-1]
                    mid_position = (previous_position + position) / 2
                    mid_position[2] += 50
                    trajectories[path].append(mid_position)
                trajectories[path].append(position)

trajectories = {
    key: numpy.array(value, dtype=numpy.float32).transpose()
    for key,value in trajectories.items()
}
trajectories;

In [4]:
balls = {
    i: Mesh(
            SphereBufferGeometry(5, 32, 16),
            MeshStandardMaterial(color=color)
        )
    for i, color in [ ['1', 'red'], ['2', 'blue'], ['3', 'green'] ]
}

In [5]:
ballGroup = Group(children=tuple(balls.values()))

view_width=200
view_height=200
camera = PerspectiveCamera( position=[0, 0, -200], aspect=1)
scene = Scene(children=[ballGroup, camera, AmbientLight()])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

In [6]:
#group = AnimationObjectGroup([balls['1'], balls['2'], balls['3']]) # fails
#group.add(balls['1']) # fails

In [7]:
def ball_track(i, trajectory):
    times = trajectory[0,:]
    positions = trajectory[1:,:]

    return VectorKeyframeTrack(name=".children[{}].position".format(i),
        times=times,
        values=positions,
        interpolation=pythreejs.enums.InterpolationModes.InterpolateSmooth                             
        )

clip = AnimationClip(tracks=[ball_track(j, trajectories[i]) for j,i in enumerate(balls)])

mixer = AnimationMixer(scene)
action = AnimationAction(mixer, clip, ballGroup)

In [8]:
renderer

Renderer(camera=PerspectiveCamera(position=(0.0, 0.0, -200.0), projectionMatrix=(2.1445069205095586, 0.0, 0.0,…

In [9]:
action.timeScale = 1

In [10]:
action

AnimationAction(clip=AnimationClip(duration=5.4116997718811035, tracks=(VectorKeyframeTrack(interpolation='Int…